Julia code for Kitaev model.
\section{Kitaev model without symmetry}

In [5]:
using SparseArrays
using Arpack
using BenchmarkTools

function index(N1::Int, N2::Int, c::Int, r::Int, atom::Int)
    #Calculate the index of the atoms in the (r,c) unit cell
    #A-sublattice atom = 0; B-subkattice: atom = 1
    c = (c + N1) % N1 
    r = (r + N2) % N2 
    #println("(",r,",",c,")")
    n = r * N1 + c
    ind = 2 * n + 1 + atom
    return ind
end

function bits(tag::Int,i::Int, N1::Int, N2::Int)
    # Chenck the i-th element of the binary representation of tag
    SiteNum  = N1 * N2 * 2
    mask = 2^(SiteNum - i)
    n = tag & mask
    if n == mask
        return 1
    else
        return 0
    end
end
    
function StateConfig(tag::Int, N1::Int, N2::Int)
    #=The configuration of the state with tag.
    Inputs: tag: tag of a state
            N1, N2: column and row numbers       
    Outputs: binary form of tag. type: 1D array=#
    
    dim = 2^(N1 * N2 * 2)
    if tag >= dim
        return println("Error: The tag is out of range.")
    else
        b = string(tag, base = 2, pad = N1 * N2 * 2)
        return b
    end
end

function flip(tag::Int, i::Int, j::Int, N1::Int, N2::Int)
    #=Flip the spin on i,j site.
    Inputs: tag: tag of a state
            N1, N2: column and row numbers
            i,j: position of spins that are flipped
    Output: The tag of new state, type: int =#
    SiteNum = N1 * N2 * 2
    f = 2^(SiteNum - i) + 2^(SiteNum - j)
    return ntag = xor(tag,f)
end

function KitaevRhom(J::Array{Float64,1}, N1::Int, N2::Int)
    #=Kitaev Hamiltonian on N1 * N2 * 2 lattice
    Inputes: N1, N2: column and row numbers
             J: coupling constants
    Outpus: The Non-zero Hamiltonian element H[i] is in the 
            position (row[i], col[i])
            H, row, col : type: list =#
    SiteNum = N1 * N2 * 2
    dim = 2^SiteNum
    
    Ham = spzeros(Float64, dim, dim)

    for tag = 0: dim-1

        for r = 0: N2-1
            for c = 0: N1-1
                ind = index(N1, N2, c, r, 0)
                
                # Sx-Sx
                next = index(N1, N2, c, r, 1)
                ntag = flip(tag, ind, next, N1, N2)
                Ham[ntag+1, tag+1] += -J[1] /4
                
                # Sy-Sy
                next = index(N1, N2, c - 1, r, 1) 
                ntag = flip(tag, ind, next, N1, N2) 
                Ham[ntag+1, tag+1] += J[2] * (bits(tag,ind, N1, N2) - 0.5) * (bits(tag,next, N1, N2) - 0.5)
                        
                #Sz-Sz
                next = index(N1, N2, c, r - 1, 1)
                Ham[tag+1, tag+1] += - J[3] * (bits(tag,ind, N1, N2) - 0.5) * (bits(tag,next, N1, N2) - 0.5)
            end
        end                       
    end
    return Ham
end

KitaevRhom (generic function with 1 method)

In [6]:
@time begin
    N1 = 2
    N2 = 3
    J = -[1.0,1.0,1.0]

    H = KitaevRhom(J, N1, N2)    
    e,x = eigs(H, which = :SR)
    println(e/(N1*N2*2))
end

[-0.204173, -0.204173, -0.204173, -0.204173, -0.195232, -0.195232]
  1.703084 seconds (5.85 M allocations: 285.850 MiB, 7.87% gc time)


In [9]:
using BenchmarkTools
N1 = 2
N2 = 3
J = [1.0,1.0,1.0]
@benchmark KitaevRhom(J, N1, N2)

BenchmarkTools.Trial: 
  memory estimate:  2.03 MiB
  allocs estimate:  35
  --------------
  minimum time:     15.214 ms (0.00% GC)
  median time:      16.616 ms (0.00% GC)
  mean time:        16.860 ms (0.00% GC)
  maximum time:     21.858 ms (0.00% GC)
  --------------
  samples:          291
  evals/sample:     1

\section{Kitaev model with translation symmetry}

In [235]:
using SparseArrays
using Arpack
using BenchmarkTools

function index(N1::Int, N2::Int, c::Int, r::Int, atom::Int)
    # Calculate the index of the atoms in the (r,c) unit cell
    # A-sublattice atom = 0; B-subkattice: atom = 1
    # index starts from 1
    c = (c + N1) % N1 
    r = (r + N2) % N2 
    #println("(",r,",",c,")")
    n = r * N1 + c
    ind = 2 * n + 1 + atom
    return ind
end

function bits(tag::Int,i::Int, N1::Int, N2::Int)
    # Chenck the i-th element of the binary representation of tag
    # i starts form 1: left to right
    SiteNum  = N1 * N2 * 2
    mask = 2^(SiteNum - i)
    n = tag & mask
    if n == mask
        return 1
    else
        return 0
    end
end

function flip(tag::Int, i::Int, j::Int, N1::Int, N2::Int)
    #=Flip the spin on i,j site.
    Inputs: tag: tag of a state
            N1, N2: column and row numbers
            i,j: position of spins that are flipped, start form 1: left->right
    Output: The tag of new state, type: int =#
    SiteNum = N1 * N2 * 2
    f = 2^(SiteNum - i) + 2^(SiteNum - j)
    return ntag = xor(tag,f)
end

function cyclebits(tag::Int, n1::Int, n2::Int, N1::Int, N2::Int)
    #=Performs a cyclic permutations of n1 steps to the right and n2 steps upward.
       Inputs: tag: tag of the reference state
               n1: cycle times of n1-direction, columns,if n1=0, no cycle in columns
               n2: cycle times of n2-direction, rows,if n2=0, no cycle in rows
               N1: number of columns
               N2: number of rows
       Outputs: ntag: the tag of the outcoming state=#
    SiteNum = N1 * N2 * 2
    ntag = 0
    for r = 0: N2-1
        for c = 0: N1-1
            for atom = 0: 1
                ind = index(N1, N2, c, r, atom)
                # notice that c,r should be positive in julia, -1%3 = -1 instead of 2
                nind = index(N1, N2, c + n1, r + n2, atom) # n1: ->right, n2 -> up, c
                ntag += 2^(SiteNum - nind) * bits(tag, ind, N1, N2)
            end
        end
    end
    return ntag
end

function findindex(tag::Int, mb::Array{Int,1})
    # find the index of a tag in mb
    # binary search
    dim = length(mb)
    low = 1
    high = dim   
    while low <= high
        mid = low + div(high - low, 2)
        if tag < mb[mid]
            high = mid -1
        elseif tag > mb[mid]
            low = mid + 1
        elseif tag == mb[mid]
            return mid
        end
    end
    if low > high
        return -1
    end
end

function MomBasis(k1::Int, k2::Int, N1::Int, N2::Int)
    #=Momentum basis list of a given k1, k2
       Inputs: k1: momentum in n1-direction
               k2: momentum in n2-direction
               N1: number of columns
               N2: number of rows
       Outpus: mbasis: momentum basis list
               Ra: list of corresponding cycle period 
                   R1: n1-direction 
                   R2: n2-direction=#
    dim = 2^(N1 * N2 * 2)
    ra = - ones(Int, dim)
    mb = [x for x = 1: dim]
        
    # Translation symmetry along n1-direction
    for tag = 0: (dim - 1)
        if ra[tag + 1] == -1
            for n1 = 1: N1
                ntag = cyclebits(tag, n1, 0, N1, N2)
                if ntag > tag
                    ra[ntag + 1] = 0
                elseif ntag == tag
                    if k1%(N1/n1) == 0
                        ra[ntag + 1] = n1
                    else
                        ra[ntag + 1] = 0
                    end
                    break
                #elseif ntag < tag
                    #if r1[ntag + 1] < 0
                        #println("missing one element at n1!")
                    #end
                end
            end
        end
    end
    
    # Translation symmetry along n2-direction
    for tag = 1 : (dim - 1)
        if ra[tag + 1] != 0
            for n2 = 1: N2
                ntag = cyclebits(tag, 0, n2, N1, N2)
                if ra[ntag + 1] != 0
                    if ntag > tag
                        ra[ntag + 1] = 0
                    elseif ntag == tag
                        if k2%(N2/n2) == 0
                            ra[ntag + 1] *= n2
                        else
                            ra[ntag + 1] *= 0
                        end
                        break
                    #elseif ntag + 1 < tag
                        #if r2[nind] < 0
                            #println("missing one element at n2!")
                        #end
                    end
                end
            end
        end
    end
    
    for i = 1: length(mb)
        mb[i] = mb[i] * sign(ra[i])
    end 
    
    ra = filter!(x->x != 0, ra)
    mb = filter!(x->x != 0, mb)
    mb = replace!(x -> x-1, mb)
    return mb, ra
end

MomBasis (generic function with 1 method)

In [245]:
using BenchmarkTools
N1 = 3
N2 = 4
k1 = 0
k2 = 0
@benchmark mb, ra = MomBasis(k1, k2, N1, N2)
#println("Finish!")
#println(length(mb))
#println(ra)

BenchmarkTools.Trial: 
  memory estimate:  384.00 MiB
  allocs estimate:  7
  --------------
  minimum time:     14.435 s (1.03% GC)
  median time:      14.435 s (1.03% GC)
  mean time:        14.435 s (1.03% GC)
  maximum time:     14.435 s (1.03% GC)
  --------------
  samples:          1
  evals/sample:     1

\section{Hamiltonian matrix}

In [192]:
# Hamiltonian matrix
using SparseArrays

function findstate(tag::Int, mb::Array{Int,1}, N1::Int, N2::Int)
    #=Find the reference state of an arbitrary state.
       Inputs: tag :tag of the input state
               mb: list of momentum basis
               t1, t2: Translation matrix
       Outputs: ntag: tag of the corresponding reference state
                l : translation steps
                ntag and l will return negative values if tag is not in mbasis.=#     
    l1 = -1
    l2 = -1
    count = 0
    for n1 = 1: N1
        for n2 = 1: N2
            ntag = cyclebits(tag, n1, n2, N1, N2)
            if ntag in mb
                l1 = n1 % N1
                l2 = n2 % N2
                break
            end
            count += 1
        end
        if ntag in mb
            break
        end
    end
    
    if count >= N1 * N2
        ntag = -1
    end
    return ntag, l1, l2
end
                           
function KitaevH(J::Array{Float64,1}, N1::Int, N2::Int, k1::Int, k2::Int, mb::Array{Int,1}, ra::Array{Int,1})
    #=Kitaev Hamiltonian on N1 * N2 * 2 lattice for a given k (with translation symmetry)
       N1: number of unit cells in n1-direction, column number
       N2: number of unit cells in n1-direction, row number
    
       Inputes: 
           N1, N2: width and length of the rhomboid cluster
           k1, k2: momentund
        
       Outputs: The Non-zero Hamiltonian element H[i] is in the 
                position (row[i], col[i])
                H, row, col : type: list
        =#
    #mb, ra = MomBasis(k1, k2, N1, N2)
    dim = length(mb)
    f1 = - 2 * pi / N1
    f2 = - 2 * pi / N2
    
    #Ham = spzeros(Complex, dim, dim)
    H_real = spzeros(Float64, dim, dim)
    #H_img = spzeros(Float64, dim, dim)
    
    for tag in mb
        i = findindex(tag, mb)
        for r = 0: N2-1
            for c = 0: N1-1
                si = index(N1, N2, c, r, 0) # start index
                
                # Sx-Sx
                ni = index(N1, N2, c, r, 1) # next index
                ntag = flip(tag, si, ni, N1, N2)
                rtag, l1, l2 = findstate(ntag, mb, N1, N2)
                if rtag > 0
                    j = findindex(rtag, mb)
                    phase_real = cos(f1*k1*l1 + f2*k2*l2) * sqrt(ra[i]/ra[j])
                    #phase_img = sin(f1*k1*l1 + f2*k2*l2) * sqrt(ra[i]/ra[j])
                    H_real[j, i] += -J[1]/4 * phase_real
                    #H_img[j, i] += -J[1]/4 * phase_img
                end
             
                # Sy-Sy
                ni = index(N1, N2, c - 1, r, 1)
                ntag = flip(tag, si, ni, N1, N2)
                rtag, l1, l2 = findstate(ntag, mb, N1, N2)
                if rtag > 0
                    j = findindex(rtag, mb)
                    phase_real = cos(f1*k1*l1 + f2*k2*l2) * sqrt(ra[i]/ra[j])
                    #phase_img = sin(f1*k1*l1 + f2*k2*l2) * sqrt(ra[i]/ra[j])
                    H_real[j, i] += J[2]*(bits(tag, si, N1, N2)-0.5)*(bits(tag, ni, N1, N2)-0.5) * phase_real
                    #H_img[j, i] += J[2]*(bits(tag, si, N1, N2)-0.5)*(bits(tag, ni, N1, N2)-0.5) * phase_img
                end
                                    
                # Sz-Sz
                ni = index(N1, N2, c, r - 1, 1)
                H_real[i, i] += - J[3] * (bits(tag, si, N1, N2)-0.5)*(bits(tag, ni, N1, N2)-0.5)
            end
        end
    end
    return H_real
end

KitaevH (generic function with 2 methods)

In [247]:
using SparseArrays
using Arpack
using BenchmarkTools

N1 = 3
N2 = 3
k1 = 0
k2 = 0
SiteNum = N1 * N2 * 2
J = -[1.0,1.0,1.0]

mb,ra = MomBasis(k1, k2, N1, N2)

@benchmark H = KitaevH(J, N1, N2, k1, k2, mb, ra)

BenchmarkTools.Trial: 
  memory estimate:  34.45 MiB
  allocs estimate:  45
  --------------
  minimum time:     99.923 s (0.01% GC)
  median time:      99.923 s (0.01% GC)
  mean time:        99.923 s (0.01% GC)
  maximum time:     99.923 s (0.01% GC)
  --------------
  samples:          1
  evals/sample:     1

In [197]:
using SparseArrays
using Arpack
using BenchmarkTools

N1 = 2
N2 = 3
k1 = 0
k2 = 0
SiteNum = N1 * N2 * 2
J = -[1.0,1.0,1.0]

mb,ra = MomBasis(k1, k2, N1, N2)

@benchmark H = KitaevH(J, N1, N2, k1, k2, mb, ra)

BenchmarkTools.Trial: 
  memory estimate:  521.77 KiB
  allocs estimate:  30
  --------------
  minimum time:     26.121 ms (0.00% GC)
  median time:      26.608 ms (0.00% GC)
  mean time:        27.144 ms (0.00% GC)
  maximum time:     32.141 ms (0.00% GC)
  --------------
  samples:          184
  evals/sample:     1

In [240]:
using SparseArrays
using Arpack

N1 = 2
N2 = 3
k1 = 0
k2 = 0
SiteNum = N1 * N2 * 2
J = -[1.0,1.0,1.0]
@time begin
mb, ra = MomBasis(k1, k2, N1, N2)
#println(mb)
#for k1 = 0: N1-1
    #for k2 = 0: N2-1
        H = KitaevH(J, N1, N2, k1, k2, mb, ra)
        e, x = eigs(H, which = :SR)
        println(real(e/SiteNum))
    #end
#end
end

[-0.204173, -0.204111, -0.195232, -0.194739, -0.193645, -0.193645]
  0.048424 seconds (1.84 k allocations: 1.254 MiB)
